In [70]:
import pandas as pd
from tabulate import tabulate

In [71]:
df = pd.read_csv('C:/Users/Thalisson/Documents/GitHub/inaFaceAnalyzer/output/catraca1M_20F2D.csv')
print(tabulate(df, headers='keys', tablefmt='pretty'))

# frame: o caminho completo para a imagem usada
# bbox: (esquerda, topo, direita, baixo) a caixa delimitadora do rosto no quadro da imagem
# detect_conf: a estimativa de confiança da detecção de rosto (dependendo do método de detecção de rosto utilizado)
# sex_decfunc: saída bruta do classificador de gênero: valores positivos são usados para homens e valores negativos para mulheres
# sex_label: predição do classificador de gênero: 'm' para homens e 'w' para mulheres
# age_decfunc: saída bruta da regressão de idade baseada nas categorias de idade do FairFace.
# 0 para (0-3 anos), 1 para (4-9 anos), 2 para (10-19 anos), 3 para (20-29 anos), etc...
# age_label: predição de idade "human readable"

+------+-------+------------------------+---------+--------------------+--------------------+---------------+-------------+-----------+--------------------+-----------------+-----------------+---------------+--------------------+
|      | frame |          bbox          | face_id |    detect_conf     |     track_conf     |  sex_decfunc  | age_decfunc | sex_label |     age_label      | sex_decfunc_avg | age_decfunc_avg | sex_label_avg |   age_label_avg    |
+------+-------+------------------------+---------+--------------------+--------------------+---------------+-------------+-----------+--------------------+-----------------+-----------------+---------------+--------------------+
|  0   |   0   |  (437, 84, 471, 118)   |    0    | 0.9998424649238586 |        nan         |  0.96975315   |  4.3312063  |     m     | 38.312063217163086 |    1.2736757    |    3.635048     |       m       | 31.35047912597656  |
|  1   |   0   |  (521, 121, 555, 155)  |    1    | 0.9997656345367432 |        

In [72]:
df = df.drop_duplicates(subset=['face_id'], keep='first')
df.reset_index(drop=True, inplace=True)
print(tabulate(df, headers='keys', tablefmt='pretty'))

# frame: o caminho completo para a imagem usada
# bbox: (esquerda, topo, direita, baixo) a caixa delimitadora do rosto no quadro da imagem
# detect_conf: a estimativa de confiança da detecção de rosto (dependendo do método de detecção de rosto utilizado)
# sex_decfunc: saída bruta do classificador de gênero: valores positivos são usados para homens e valores negativos para mulheres
# sex_label: predição do classificador de gênero: 'm' para homens e 'w' para mulheres
# age_decfunc: saída bruta da regressão de idade baseada nas categorias de idade do FairFace.
# 0 para (0-3 anos), 1 para (4-9 anos), 2 para (10-19 anos), 3 para (20-29 anos), etc...
# age_label: predição de idade "human readable"

+-----+-------+------------------------+---------+--------------------+------------+---------------+-------------+-----------+--------------------+-----------------+-----------------+---------------+--------------------+
|     | frame |          bbox          | face_id |    detect_conf     | track_conf |  sex_decfunc  | age_decfunc | sex_label |     age_label      | sex_decfunc_avg | age_decfunc_avg | sex_label_avg |   age_label_avg    |
+-----+-------+------------------------+---------+--------------------+------------+---------------+-------------+-----------+--------------------+-----------------+-----------------+---------------+--------------------+
|  0  |   0   |  (437, 84, 471, 118)   |    0    | 0.9998424649238586 |    nan     |  0.96975315   |  4.3312063  |     m     | 38.312063217163086 |    1.2736757    |    3.635048     |       m       | 31.35047912597656  |
|  1  |   0   |  (521, 121, 555, 155)  |    1    | 0.9997656345367432 |    nan     |  0.29838723   |  3.4812188  |  

In [73]:
def calculate_iou(boxA, boxB):
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    
    # area de inters
    interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
    
    # area de 2 bbox
    boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
    boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
    
    # area de uniao
    unionArea = boxAArea + boxBArea - interArea
    
    iou = interArea / float(unionArea)
    return iou

In [74]:

iou_threshold = 0.2

# coluna para verificar os ids de face q sao iguais
df['same_face_id'] = -1
ious = []
for i in range(len(df)):
    for j in range(i + 1, len(df)):
        boxA = eval(df.loc[i, 'bbox'])
        boxB = eval(df.loc[j, 'bbox'])
        iou = calculate_iou(boxA, boxB)
        ious.append(iou)
        if iou > iou_threshold:
            if df.loc[i, 'same_face_id'] == -1 and df.loc[j, 'same_face_id'] == -1:
                df.loc[i, 'same_face_id'] = df.loc[i, 'face_id']
                df.loc[j, 'same_face_id'] = df.loc[i, 'face_id']
            elif df.loc[i, 'same_face_id'] != -1:
                df.loc[j, 'same_face_id'] = df.loc[i, 'same_face_id']
            elif df.loc[j, 'same_face_id'] != -1:
                df.loc[i, 'same_face_id'] = df.loc[j, 'same_face_id']

# se ainda for -1, nao teve nenhuma correspodencia, entao setamos com o proprio id.
df.loc[df['same_face_id'] == -1, 'same_face_id'] = df['face_id']



In [75]:
print(tabulate(df, headers='keys', tablefmt='pretty'))
# print(ious)


# filtered_df = df[df['same_face_id'] == 0]
# print(tabulate(filtered_df, headers='keys', tablefmt='pretty'))

+-----+-------+------------------------+---------+--------------------+------------+---------------+-------------+-----------+--------------------+-----------------+-----------------+---------------+--------------------+--------------+
|     | frame |          bbox          | face_id |    detect_conf     | track_conf |  sex_decfunc  | age_decfunc | sex_label |     age_label      | sex_decfunc_avg | age_decfunc_avg | sex_label_avg |   age_label_avg    | same_face_id |
+-----+-------+------------------------+---------+--------------------+------------+---------------+-------------+-----------+--------------------+-----------------+-----------------+---------------+--------------------+--------------+
|  0  |   0   |  (437, 84, 471, 118)   |    0    | 0.9998424649238586 |    nan     |  0.96975315   |  4.3312063  |     m     | 38.312063217163086 |    1.2736757    |    3.635048     |       m       | 31.35047912597656  |      0       |
|  1  |   0   |  (521, 121, 555, 155)  |    1    | 0.999

In [79]:
grouped_df = df.groupby('same_face_id').agg({
    'sex_decfunc_avg': 'mean',  # media do sex_decfunc_avg
    'age_label_avg': 'mean'   # media do age_decfunc_avg
}).reset_index()

# print(tabulate(grouped_df, headers='keys', tablefmt='pretty'))
# grouped_df.to_csv('C:/Users/Thalisson/Documents/GitHub/inaFaceAnalyzer/output/catraca1M_20F2D_iou02.csv', index=False)

df = pd.read_csv('C:/Users/Thalisson/Documents/GitHub/inaFaceAnalyzer/output/catraca1M_20F1D_iou02.csv')
print(tabulate(df, headers='keys', tablefmt='pretty'))

+----+--------------+---------------------+--------------------+
|    | same_face_id |   sex_decfunc_avg   |   age_label_avg    |
+----+--------------+---------------------+--------------------+
| 0  |     0.0      | -0.0771049558920948 | 26.28199925017451  |
| 1  |     1.0      | 0.0345355944117647  | 24.62569029892192  |
| 2  |     2.0      | -0.7945396810714285 | 28.254024585088093 |
| 3  |     3.0      |     -2.5403866      | 32.80825734138489  |
| 4  |     6.0      | -0.9213134722222224 | 22.41381704807281  |
| 5  |     37.0     |      1.2148228      | 27.156835794448853 |
| 6  |     41.0     |     2.05966132      | 28.732765674591064 |
| 7  |     48.0     | 1.5219525430769232  | 26.959977516761192 |
| 8  |     68.0     |     -1.9579633      | 32.093799114227295 |
| 9  |     75.0     |     0.310973765     | 28.153598308563232 |
| 10 |     92.0     | -0.2966012948461538 | 27.799566158881557 |
| 11 |     98.0     |  0.280194414974359  | 24.62899963061015  |
| 12 |    100.0     | -1.